In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import numpy as np
import scanpy as sc
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import time

# Evaluation metrics
from sklearn.metrics import (homogeneity_score,v_measure_score,adjusted_mutual_info_score,normalized_mutual_info_score,adjusted_rand_score,fowlkes_mallows_score)

# Import SpaLP modules    Python package: https://github.com/dbjzs/SpaLP
from SpaLP.LP import SpatialLocalPooling
from SpaLP.utils import set_seed, prepare_inputs
# Set random seed
set_seed(7)

#Check the GPU memory and CPU memory
import memory_profiler  # pip install memory_profiler
import torch
import gc

# Check device
import cpuinfo   # pip install py-cpuinfo
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if device.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")
info = cpuinfo.get_cpu_info()
print("CPU:", info['brand_raw'])

Using device: cuda
GPU: NVIDIA A800-SXM4-80GB
CPU: Intel(R) Xeon(R) Platinum 8462Y+


In [2]:
#Check the GPU memory and CPU memory
from memory_profiler import memory_usage
def measure_resources(func):
    def wrapper(*args, **kwargs):
        gc.collect()
        torch.cuda.synchronize()
        torch.cuda.reset_peak_memory_stats()
        torch.cuda.empty_cache()
        
        before = memory_usage(-1, interval=0.1, timeout=1)[0]
        mem_peak, result = memory_usage((func, args, kwargs),max_usage=True,retval=True)
        print(f"CPU peak memory: {(mem_peak - before)/1024:.2f} GB")
        
        torch.cuda.synchronize()
        gpu_peak = torch.cuda.max_memory_allocated() / 1024**3
        print(f"GPU peak memory: {gpu_peak:.2f} GB")
        return result
    return wrapper

In [3]:
def train_spalp(graph, in_channels, hidden_dim, epochs, lr, device,seed):
    """Train SpaLP model and return embeddings."""
    from tqdm.auto import tqdm
    set_seed(seed)
    pbar = tqdm(range(epochs), desc="Training",ncols=200)
    model = SpatialLocalPooling(in_channels, hidden_dim).to(device)
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    model.train()
    losses = []
    
    for epoch in pbar:
        start_time = time.time()
        model.train()
        optimizer.zero_grad()
        reconstructed,embedding = model(graph.features, graph.neighbor_idx)
        loss = criterion(reconstructed, graph.features)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        elapsed = time.time() - start_time
        pbar.set_postfix({"Epoch": epoch,"Loss": f"{loss.item():.4f}"})
    
    # Get final embeddings
    model.eval()
    with torch.no_grad():
        reconstructed,embedding = model(graph.features, graph.neighbor_idx)
        reconstructed = reconstructed.cpu().numpy()
        embedding=embedding.cpu().numpy()
    return embedding, losses

# Dataset 1: Low quality CosMx Human Kidney Cancer data (1,236,281 cells)

In [5]:
# Load CosMx Human Kidney Cancer data
adata = sc.read_h5ad('/home/dbj/SpaLA/CosMxKidneycancer/cosmx_kidney.h5ad')
adata.var_names_make_unique()
print(f"Shape: {adata.shape[0]:,} cells x {adata.shape[1]:,} genes")
print(f"obsm keys: {list(adata.obsm.keys())}")

Shape: 1,236,281 cells x 1,000 genes
obsm keys: ['spatial']


In [6]:
# Preprocessing standardization  *******
start_time = time.time()
sc.pp.normalize_total(adata, inplace=True)
sc.pp.log1p(adata)
sc.pp.scale(adata)
adata.obsm['feat']=adata.X
prep_time = time.time() - start_time

print(f"Spatial coords shape: {adata.obsm['spatial'].shape}")
print(f"Preprocessing standardization: {prep_time:.2f}s")

Spatial coords shape: (1236281, 2)
Preprocessing standardization: 8.85s


In [7]:
# Set the hyperparameters  k=4
k=4
in_channels = adata.obsm['feat'].shape[1]
lr=0.001
epochs = 200
hidden_dim=64
print(f"Feature dimensions: {in_channels}")

Feature dimensions: 1000


In [8]:
# Build neighbor graph  k=4
start_time = time.time()
graph = prepare_inputs(adata, k=k,device=device)
prep_time = time.time() - start_time
print(f"Graph preparation: {prep_time:.2f}s")

Graph preparation: 1.56s


In [9]:
%%time
#runtime /GPU peak memory/CPU peak memory
start_time = time.time()
@measure_resources
def run_train():
    return train_spalp(graph, in_channels, hidden_dim, epochs, lr, device, seed=7)#The default random seed for the SpaLP algorithm is 7.
embedding, losses = run_train()
train_time = time.time() - start_time
print(f"Training time: {train_time:.2f}s")

# Store embeddings
adata.obsm['SpaLP'] = embedding

Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [01:03<00:00,  3.15it/s, Epoch=199, Loss=0.9836]


CPU peak memory: 4.85 GB
GPU peak memory: 30.15 GB
Training time: 67.80s
CPU times: user 1min 3s, sys: 2.85 s, total: 1min 6s
Wall time: 1min 7s


In [12]:
%%time
# Clustering and UMAP
start_time = time.time()
sc.pp.neighbors(adata, use_rep="SpaLP")
sc.tl.leiden(adata, resolution=0.2, random_state=2024, key_added="SpaLP")#The clustering results save in adata.obs['SpaLP']
elapsed = time.time() - start_time
print(f"Leiden time: {int(elapsed//60)}min {elapsed%60:.2f}s")

t0 = time.time()
sc.tl.umap(adata)
elapsed = time.time() - t0
print(f"UMAP time: {int(elapsed//60)}min {elapsed%60:.2f}s")
print(f"Found {len(adata.obs['SpaLP'].unique())} clusters")

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


Leiden time: 150min 27.31s


IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


UMAP time: 16min 16.79s
Found 7 clusters
CPU times: user 6h 13min 7s, sys: 2h 26min 10s, total: 8h 39min 18s
Wall time: 2h 46min 44s


In [10]:
%%time
# KMeans time
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=15, random_state=2024) 
labels = kmeans.fit_predict(adata.obsm['SpaLP'])
adata.obs['Kmeans_SpaLP']=labels.astype(str)  #The clustering results save in adata.obs['Kmeans_SpaLP']

CPU times: user 58.7 s, sys: 2min 5s, total: 3min 3s
Wall time: 3.76 s


## Obtaining the Leiden and UMAP results using rapid single cell (GPU-scanpy)

In [11]:
adata.write_h5ad('/home/dbj/SpaLP/gittest/repoduced/result/Fig6Low—quality-CosMx-Human-Kidney-Cancer.h5ad',compression='gzip')  #Save the embedding for rapid single-cell (GPU-scanpy) analysis

# Dataset 2: Xenium Gastric cancer data (696,314 cells)

In [4]:
adata=sc.read_h5ad('/home/dbj/SpaLA/Gastric_cancer/Gastric.h5ad')
adata.var_names_make_unique()
print(f"Shape: {adata.shape[0]:,} cells x {adata.shape[1]:,} genes")
print(f"obsm keys: {list(adata.obsm.keys())}")

Shape: 696,314 cells x 377 genes
obsm keys: ['spatial']


In [5]:
# Preprocessing standardization  *******
start_time = time.time()
sc.pp.filter_cells(adata,min_genes=2)
sc.pp.normalize_total(adata,inplace=True)
sc.pp.log1p(adata)
sc.pp.scale(adata)
adata.obsm['feat']=adata.X
prep_time = time.time() - start_time

print(f"Spatial coords shape: {adata.obsm['spatial'].shape}")
print(f"Preprocessing standardization: {prep_time:.2f}s")

Spatial coords shape: (662434, 2)
Preprocessing standardization: 2.18s


In [6]:
# Set the hyperparameters  k=5
k=5
in_channels = adata.obsm['feat'].shape[1]
lr=0.001
epochs = 200
hidden_dim=64
print(f"Feature dimensions: {in_channels}")

Feature dimensions: 377


In [7]:
# Build neighbor graph  k=5
start_time = time.time()
graph = prepare_inputs(adata, k=k,device=device)
prep_time = time.time() - start_time
print(f"Graph preparation: {prep_time:.2f}s")

Graph preparation: 0.56s


In [8]:
%%time
#runtime /GPU peak memory/CPU peak memory
start_time = time.time()
@measure_resources
def run_train():
    return train_spalp(graph, in_channels, hidden_dim, epochs, lr, device, seed=7)#The default random seed for the SpaLP algorithm is 7.
embedding, losses = run_train()
train_time = time.time() - start_time
print(f"Training time: {train_time:.2f}s")

# Store embeddings
adata.obsm['SpaLP'] = embedding

Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:37<00:00,  5.39it/s, Epoch=199, Loss=0.9581]


CPU peak memory: 1.27 GB
GPU peak memory: 13.41 GB
Training time: 38.99s
CPU times: user 37.1 s, sys: 919 ms, total: 38.1 s
Wall time: 39 s


In [9]:
%%time
# Clustering and UMAP
start_time = time.time()
sc.pp.neighbors(adata, use_rep="SpaLP")
sc.tl.leiden(adata, resolution=0.2, random_state=2024, key_added="SpaLP")#The clustering results save in adata.obs['SpaLP']
elapsed = time.time() - start_time
print(f"Leiden time: {int(elapsed//60)}min {elapsed%60:.2f}s")

t0 = time.time()
sc.tl.umap(adata)
elapsed = time.time() - t0
print(f"UMAP time: {int(elapsed//60)}min {elapsed%60:.2f}s")
print(f"Found {len(adata.obs['SpaLP'].unique())} clusters")

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


Leiden time: 63min 29.93s
UMAP time: 7min 22.29s
Found 5 clusters
CPU times: user 2h 21min 40s, sys: 55min 30s, total: 3h 17min 10s
Wall time: 1h 10min 52s


In [10]:
%%time
# KMeans time
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=15, random_state=2024) 
labels = kmeans.fit_predict(adata.obsm['SpaLP'])
adata.obs['Kmeans_SpaLP']=labels.astype(str)  #The clustering results save in adata.obs['Kmeans_SpaLP']

CPU times: user 47.3 s, sys: 1min 18s, total: 2min 6s
Wall time: 2.64 s


## Obtaining the Leiden and UMAP results using rapid single cell (GPU-scanpy)

In [11]:
adata.write_h5ad('/home/dbj/SpaLP/gittest/repoduced/result/Fig6Xenium_Gastric_cancer.h5ad',compression='gzip')  #Save the embedding for rapid single-cell (GPU-scanpy) analysis

# Dataset 3: Xenium Milti-omics cancer data (465,545 cells)

In [12]:
adata1=sc.read_h5ad('/home/dbj/SpaLA/Xeniummultiomics/RNA.h5ad')
adata2=sc.read_h5ad('/home/dbj/SpaLA/Xeniummultiomics/Protein.h5ad')
adata1,adata2

(AnnData object with n_obs × n_vars = 465545 × 396
     obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'TACCO_celltype'
     var: 'gene_name', 'feature_type'
     obsm: 'spatial',
 AnnData object with n_obs × n_vars = 465545 × 27
     obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'genomic_control_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'nucleus_count', 'segmentation_method', 'TACCO_celltype'
     var: 'gene_name', 'feature_type'
     obsm: 'spatial')

In [13]:
# Preprocessing standardization  *******
start_time = time.time()
sc.pp.normalize_total(adata1,inplace=True)
sc.pp.log1p(adata1)
sc.pp.scale(adata1)
adata1.obsm['feat']=adata1.X
sc.pp.normalize_total(adata2,inplace=True)
sc.pp.log1p(adata2)
sc.pp.scale(adata2)
adata2.obsm['feat']=adata2.X

adata=sc.AnnData(np.hstack([adata1.obsm['feat'],adata2.obsm['feat']]))
sc.pp.scale(adata)
adata.var_names = adata1.var_names.tolist() + adata2.var_names.tolist()
rna_len = adata1.var_names.shape[0]
prot_len = adata2.var_names.shape[0]
adata.var['feature_type'] = ['RNA'] * rna_len + ['Protein'] * prot_len
adata.obs_names=adata2.obs_names
adata.obsm['spatial']=adata2.obsm['spatial']
adata.obsm['feat']=adata.X
prep_time = time.time() - start_time

print(f"Spatial coords shape: {adata.obsm['spatial'].shape}")
print(f"Preprocessing standardization: {prep_time:.2f}s")

Spatial coords shape: (465545, 2)
Preprocessing standardization: 2.59s


In [14]:
# Set the hyperparameters  k=4
k=4
in_channels = adata.obsm['feat'].shape[1]
lr=0.001
epochs = 200
hidden_dim=64
print(f"Feature dimensions: {in_channels}")

Feature dimensions: 423


In [15]:
# Build neighbor graph  k=4
start_time = time.time()
graph = prepare_inputs(adata, k=k,device=device)
prep_time = time.time() - start_time
print(f"Graph preparation: {prep_time:.2f}s")

Graph preparation: 0.34s


In [16]:
%%time
#runtime /GPU peak memory/CPU peak memory
start_time = time.time()
@measure_resources
def run_train():
    return train_spalp(graph, in_channels, hidden_dim, epochs, lr, device, seed=7)#The default random seed for the SpaLP algorithm is 7.
embedding, losses = run_train()
train_time = time.time() - start_time
print(f"Training time: {train_time:.2f}s")

# Store embeddings
adata.obsm['SpaLP'] = embedding

Training: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:27<00:00,  7.27it/s, Epoch=199, Loss=0.9423]


CPU peak memory: 0.69 GB
GPU peak memory: 7.89 GB
Training time: 29.70s
CPU times: user 28 s, sys: 767 ms, total: 28.7 s
Wall time: 29.7 s


In [17]:
%%time
# Clustering and UMAP
start_time = time.time()
sc.pp.neighbors(adata, use_rep="SpaLP")
sc.tl.leiden(adata, resolution=0.2, random_state=2024, key_added="SpaLP")#The clustering results save in adata.obs['SpaLP']
elapsed = time.time() - start_time
print(f"Leiden time: {int(elapsed//60)}min {elapsed%60:.2f}s")

t0 = time.time()
sc.tl.umap(adata)
elapsed = time.time() - t0
print(f"UMAP time: {int(elapsed//60)}min {elapsed%60:.2f}s")
print(f"Found {len(adata.obs['SpaLP'].unique())} clusters")

IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


Leiden time: 23min 17.02s


IOStream.flush timed out
IOStream.flush timed out
IOStream.flush timed out


UMAP time: 4min 37.49s
Found 7 clusters
CPU times: user 1h 8min, sys: 30min 6s, total: 1h 38min 7s
Wall time: 27min 54s


In [18]:
%%time
# KMeans time
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=15, random_state=2024) 
labels = kmeans.fit_predict(adata.obsm['SpaLP'])
adata.obs['Kmeans_SpaLP']=labels.astype(str)  #The clustering results save in adata.obs['Kmeans_SpaLP']

CPU times: user 43.4 s, sys: 1min 4s, total: 1min 47s
Wall time: 2.02 s


In [19]:
adata.write_h5ad('/home/dbj/SpaLP/gittest/repoduced/result/Fig6Xenium_multi-omics.h5ad',compression='gzip')  #Save the embedding for rapid single-cell (GPU-scanpy) analysis